In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('../data/training_data.csv')

/var/folders/xy/hzd4ztd10dn2_fwhxdz4hd340000gn/T/ipykernel_17464/3704762819.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/training_data.csv')


In [3]:
# choose features for the model
df = df.filter(['charge_off', 'mth_code', 'snapshot', 'account_status_code', 'active', 'bank_fico_buckets_20', 'ever_delinquent_flg', 'industry', 'mob', 'stmt_balance', 'prev_balance'])

In [4]:
# encode the date & month to ordinal values
df = df.replace(regex={201801:1,201802:2,201803:3,201804:4,201805:5,201806:6,201807:7,201808:8,
                 201809:9,201810:10,201811:11,201812:12,201901:13,201902:14,201903:15,
                 201904:16,201905:17,201906:18,201907:19,201908:20,201909:21,201910:22,
                 201911:23,201912:24})

In [5]:
# encode account status code to binary values ('A' -> 1, others -> 0)
df['account_status_code'] = np.where(df['account_status_code'] == 'A', 1, 0)

In [6]:
# create balance_decrease, month_diff columns
df["balance_decrease"] = df["prev_balance"] - df["stmt_balance"]
df["month_diff"] = df["mth_code"] - df["snapshot"]

In [7]:
# encode bank_fico_buckets_20 to ordinal values
df['bank_fico_buckets_20'] = np.where(df['bank_fico_buckets_20'] == 'Missing', '0', df['bank_fico_buckets_20'])
df['bank_fico_buckets_20'] = np.where(df['bank_fico_buckets_20'] == 'Exception', '0', df['bank_fico_buckets_20'])

enc = OrdinalEncoder(categories=[['0','<= 560','561-580','581-600','601-620','621-640','641-660','661-680','681-700','701-720','721-740','741-760', '761+']])
df[["bank_fico_buckets_20"]] = enc.fit_transform(df[["bank_fico_buckets_20"]])

In [8]:
# one-hot encode the industry categorical value
df['industry_A'] = np.where(df['industry'] == 'A', 1, 0)
df['industry_B'] = np.where(df['industry'] == 'B', 1, 0)
df['industry_C'] = np.where(df['industry'] == 'C', 1, 0)

In [9]:
df.head()

,charge_off,mth_code,snapshot,account_status_code,active,bank_fico_buckets_20,ever_delinquent_flg,industry,mob,stmt_balance,prev_balance,balance_decrease,month_diff,industry_A,industry_B,industry_C
0,0,7,1,0,1,3.0,0.0,B,3,34.77,1918.32,1883.55,6,0,1,0
1,0,18,1,0,1,10.0,0.0,C,159,0.00,0.00,0.00,17,0,0,1
2,0,17,1,0,1,12.0,0.0,A,22,0.00,0.00,0.00,16,1,0,0
3,0,6,1,0,1,12.0,1.0,B,16,0.00,0.00,0.00,5,0,1,0
4,0,14,1,1,1,10.0,0.0,B,3,0.00,0.00,0.00,13,0,1,0


In [90]:
df.to_csv('../data/training_preprocessed.csv', encoding='utf-8', index=False)

In [ ]:
##### combine macro data for account ver #####

In [11]:
macro = pd.read_csv('../data/macro_manual_selected_features.csv')

In [12]:
macro.head()

,date,month,unemployment_rate,debt_service_burden,consumer_credit,interest_rates,charge_off_rates
0,201801,1,4.0,9.772692,2.533887,4.500000,3.664279
1,201802,2,4.1,9.727890,2.552633,4.500000,3.650039
2,201803,3,4.0,9.709184,2.527467,4.579545,3.635683
3,201804,4,4.0,9.707564,2.486718,4.750000,3.627844
4,201805,5,3.8,9.707665,2.466230,4.750000,3.634638


In [13]:
macro = macro.rename(columns={"month": "mth_code"})

In [14]:
macro = macro[macro["mth_code"]<=24]

In [15]:
# join the macro data to training data with mth_code column
macro_joined = pd.merge(df, macro, how="left", on=["mth_code"])

In [16]:
macro_joined

,charge_off,mth_code,snapshot,account_status_code,active,bank_fico_buckets_20,ever_delinquent_flg,industry,mob,stmt_balance,...,month_diff,industry_A,industry_B,industry_C,date,unemployment_rate,debt_service_burden,consumer_credit,interest_rates,charge_off_rates
0,0,7,1,0,1,3.0,0.0,B,3,34.77,...,6,0,1,0,201807,3.8,9.687634,2.472498,5.000000,3.680373
1,0,18,1,0,1,10.0,0.0,C,159,0.00,...,17,0,0,1,201906,3.6,9.725124,2.470367,5.500000,3.747979
2,0,17,1,0,1,12.0,0.0,A,22,0.00,...,16,1,0,0,201905,3.6,9.706048,2.435236,5.500000,3.725027
3,0,6,1,0,1,12.0,1.0,B,16,0.00,...,5,0,1,0,201806,4.0,9.698626,2.371999,4.892857,3.657519
4,0,14,1,1,1,10.0,0.0,B,3,0.00,...,13,0,1,0,201902,3.8,9.710745,2.445232,5.500000,3.698200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5758218,0,24,24,1,1,6.0,0.0,A,41,1288.19,...,0,1,0,0,201912,3.6,9.852360,2.407747,4.750000,3.884991
5758219,0,24,24,1,1,9.0,0.0,C,47,27.08,...,0,0,0,1,201912,3.6,9.852360,2.407747,4.750000,3.884991
5758220,0,24,24,0,1,11.0,0.0,B,34,0.00,...,0,0,1,0,201912,3.6,9.852360,2.407747,4.750000,3.884991
5758221,0,24,24,0,1,8.0,0.0,B,50,662.81,...,0,0,1,0,201912,3.6,9.852360,2.407747,4.750000,3.884991


In [210]:
macro_joined.to_csv('../data/training_for_account_ver.csv', encoding='utf-8', index=False)

In [ ]:
##### group by month for ml model aggreagate ver #####

In [94]:
# find the total charge off ## per each grouped row 
group_sum = df.groupby(['snapshot', 'mth_code']).agg("sum")
group_count = df.groupby(['snapshot', 'mth_code']).agg("count")

In [95]:
# make new columns that indiciate the values of the groups 
idx = group_sum.index
snapshot_list = []
mth_code_list=[]
for i in range(len(idx)):
    snapshot_list.append(idx[i][0])
    mth_code_list.append(idx[i][1])   
group_sum.insert(0,"snapshot", snapshot_list)
group_sum.insert(1,"mth_code", mth_code_list)

In [64]:
group_sum.head()

,snapshot,mth_code,charge_off
0,1,1,0
1,1,2,78
2,1,3,95
3,1,4,67
4,1,5,81


In [96]:
# drop columns that are not needed 
group_sum = group_sum[["snapshot", "mth_code", "charge_off"]]

In [97]:
# find the average for the columns except for the charge off ##
group_avg = df.groupby(['snapshot', 'mth_code']).agg("mean")

In [98]:
group_avg = group_avg.drop(columns="charge_off")

In [99]:
group_avg

account_status_code    active  bank_fico_buckets_20  \
snapshot mth_code                                                        
1        1                    0.672928  1.000000              9.217998   
         2                    0.666902  0.996072              9.170578   
         3                    0.672330  0.995196              9.207221   
         4                    0.670054  0.996595              9.227755   
         5                    0.668672  0.995871              9.173745   
...                                ...       ...                   ...   
22       23                   0.532169  0.997353              9.373867   
         24                   0.527422  0.996828              9.394249   
23       23                   0.534353  1.000000              9.479986   
         24                   0.536012  0.997562              9.492889   
24       24                   0.524335  1.000000              9.406989   

                   ever_delinquent_flg        mob  stmt_balance  prev_balance  \
snapshot mth_code                                                               
1        1                    0.111693  60.215480    785.094246    810.826223   
         2                    0.111755  60.253425    771.162752    789.697385   
         3                    0.108515  59.255512    770.332103    796.755381   
         4                    0.112861  58.943188    775.782264    776.045289   
         5                    0.109457  59.937905    775.811367    796.240531   
...                                ...        ...           ...           ...   
22       23                   0.103176  58.953782    775.917080    786.746013   
         24                   0.098844  59.038883    771.720375    805.916063   
23       23                   0.096321  56.129763    804.717327    801.638307   
         24                   0.097115  56.736337    812.448763    825.465287   
24       24                   0.098151  56.895417    784.411571    770.155697   

                   balance_decrease  month_diff  industry_A  industry_B  \
snapshot mth_code                                                         
1        1                25.731977         0.0    0.227112    0.583795   
         2                18.534632         1.0    0.231315    0.582192   
         3                26.423278         2.0    0.226032    0.588491   
         4                 0.263025         3.0    0.232075    0.582804   
         5                20.429164         4.0    0.228601    0.582513   
...                             ...         ...         ...         ...   
22       23               10.828933         1.0    0.223455    0.616919   
         24               34.195688         2.0    0.227617    0.610099   
23       23               -3.079021         0.0    0.227959    0.609597   
         24               13.016524         1.0    0.228820    0.610880   
24       24              -14.255873         0.0    0.231096    0.611395   

                   industry_C  
snapshot mth_code              
1        1           0.189093  
         2           0.186493  
         3           0.185477  
         4           0.185121  
         5           0.188886  
...                       ...  
22       23          0.159625  
         24          0.162284  
23       23          0.162444  
         24          0.160301  
24       24          0.157508  

[300 rows x 12 columns]

In [100]:
group_sum = group_sum[["snapshot", "mth_code", "charge_off"]]

In [101]:
group_sum.head()

snapshot  mth_code  charge_off
snapshot mth_code                                
1        1                1         1           0
         2                1         2          78
         3                1         3          95
         4                1         4          67
         5                1         5          81

In [104]:
# make a combined dataset 
aggregated = pd.concat([group_sum, group_avg], axis=1)
aggregated["count"] = group_count["charge_off"]
aggregated["charge_off_p"] = aggregated["charge_off"]/aggregated["count"]
aggregated = aggregated.drop(columns={"charge_off", "count"})
aggregated = aggregated.reset_index(drop=True)

In [105]:
aggregated.head()

,snapshot,mth_code,account_status_code,active,bank_fico_buckets_20,ever_delinquent_flg,mob,stmt_balance,prev_balance,balance_decrease,month_diff,industry_A,industry_B,industry_C,charge_off_p
0,1,1,0.672928,1.000000,9.217998,0.111693,60.215480,785.094246,810.826223,25.731977,0.0,0.227112,0.583795,0.189093,0.000000
1,1,2,0.666902,0.996072,9.170578,0.111755,60.253425,771.162752,789.697385,18.534632,1.0,0.231315,0.582192,0.186493,0.003928
2,1,3,0.672330,0.995196,9.207221,0.108515,59.255512,770.332103,796.755381,26.423278,2.0,0.226032,0.588491,0.185477,0.004804
3,1,4,0.670054,0.996595,9.227755,0.112861,58.943188,775.782264,776.045289,0.263025,3.0,0.232075,0.582804,0.185121,0.003405
4,1,5,0.668672,0.995871,9.173745,0.109457,59.937905,775.811367,796.240531,20.429164,4.0,0.228601,0.582513,0.188886,0.004129


In [ ]:
df.to_csv('../data/training_for_aggregated_ver.csv', encoding='utf-8', index=False)